# Vehicle Detection Testing

This notebook tests the trained YOLOv8 model on example images.

In [1]:
import torch
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

## Load Model and Configuration

In [ ]:
# Load the trained model
model_path = Path('../runs/vehicle_detection/weights/best.pt')
if not model_path.exists():
    print(f"Error: Model not found at {model_path}")
    model_path = Path('yolov8n.pt')  # Use pretrained model if trained one not found
    
model = YOLO(str(model_path))

# Class names
class_names = ['Ambulance', 'Fire Engine', 'Police', 'Non Emergency']

## Define Helper Functions

In [ ]:
def plot_detection(image, results):
    """Plot detection results on the image"""
    # Create a copy of the image to draw on
    img = image.copy()
    
    for result in results:
        boxes = result.boxes  # Boxes object for bbox outputs
        
        for box in boxes:
            # Get box coordinates and confidence
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            cls = int(box.cls[0])
            
            # Draw bounding box
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            # Add label
            label = f'{class_names[cls]} {conf:.2f}'
            cv2.putText(img, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    return img

def detect_and_display(image_path):
    """Run detection on an image and display results"""
    # Read image
    img = cv2.imread(str(image_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Run inference
    results = model(img)
    
    # Plot results
    detected_img = plot_detection(img, results)
    
    # Display results
    plt.figure(figsize=(12, 8))
    plt.imshow(detected_img)
    plt.axis('off')
    plt.show()
    
    return results

## Test on Sample Images

In [ ]:
# Test directory
test_dir = Path('../Dataset/images/test')

# Get some test images
test_images = list(test_dir.glob('*.png'))[:5]  # Test first 5 images

# Run detection on each test image
for img_path in test_images:
    print(f"\nDetecting vehicles in {img_path.name}")
    results = detect_and_display(img_path)
    
    # Print detection results
    for result in results:
        for box in result.boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            print(f"Detected {class_names[cls]} with confidence {conf:.2f}")